
# Re-ConECT Project

text-based Rehabiliation AI for Continuous functional Evaluation & Customized Training

#1. Setting up the Required Environment

In [ ]:
!pip uninstall pyarrow requests
!pip install pyarrow==14.0.1 requests==2.31.0

In [ ]:
! pip3 install -qU guardrails-ai openai langchain_community langchain_experimental langchain-upstage sentence-transformers langchainhub langchain-chroma langchain matplotlib python-dotenv tavily-python ragas faiss-cpu tokenizers

In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
#@title set API key
from pprint import pprint
import os

import warnings
warnings.filterwarnings('ignore')

from IPython import get_ipython

upstage_api_key_env_name = 'UPSTAGE_API_KEY'
def load_env():
    if 'google.colab' in str(get_ipython()):
        # Running in Google Colab
        from google.colab import userdata
        upstage_api_key = userdata.get(upstage_api_key_env_name)
        return os.environ.setdefault('UPSTAGE_API_KEY', upstage_api_key)
    else:
        # Running in local Jupyter Notebook
        from dotenv import load_dotenv
        load_dotenv()
        return os.environ.get(upstage_api_key_env_name)

UPSTAGE_API_KEY = load_env()

In [ ]:
! pip3 install -qU  markdownify  langchain-upstage rank_bm25 python-dotenv

#2. Apply RAG based on specialized medical knowledge.

In [ ]:
from langchain_upstage import UpstageLayoutAnalysisLoader

pdf_files = [
    "/content/drive/MyDrive/240814_Llama_RAG/1_Neck_Pain.pdf",
    "/content/drive/MyDrive/240814_Llama_RAG/2_Neck_Pain.pdf",
    "/content/drive/MyDrive/240814_Llama_RAG/3_Neck_Pain.pdf",
    "/content/drive/MyDrive/240814_Llama_RAG/4_CUE_T_Manual.pdf",
    "/content/drive/MyDrive/240814_Llama_RAG/5_CUE_T_Score_Conversion.pdf",
    "/content/drive/MyDrive/240814_Llama_RAG/6_PTX.pdf",
    "/content/drive/MyDrive/240814_Llama_RAG/7_Stroke_Complications.pdf",
    "/content/drive/MyDrive/240814_Llama_RAG/8_Spinal_Cord_Injury_Complications.pdf"
]

loaders = []
docs = []

for i, pdf_file in enumerate(pdf_files, start=1):
    print(f"Processing file {i}: {pdf_file}")

    loader = UpstageLayoutAnalysisLoader(
        pdf_file, use_ocr=True, output_type="html"
    )
    loaders.append(loader)

    doc = loader.load()
    docs.append(doc)

    print(f"File {i} processed successfully.")

print("All files have been processed.")

In [ ]:
# Proceed with RAG using the corresponding code.

from langchain_community.retrievers import BM25Retriever
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_upstage import ChatUpstage
from typing import List, Dict

def process_documents(docs: List[List], queries: List[str]) -> Dict[int, Dict[str, str]]:
    text_splitter = RecursiveCharacterTextSplitter.from_language(
        chunk_size=1000, chunk_overlap=100, language=Language.HTML
    )

    llm = ChatUpstage()

    prompt_template = PromptTemplate.from_template(
        """
        Please provide most correct answer from the following context.
        ---
        Question: {question}
        ---
        Context: {Context}
        """
    )
    chains = prompt_template | llm | StrOutputParser()

    results = {}
    for i, doc in enumerate(docs):
        splits = text_splitter.split_documents(doc)
        retriever = BM25Retriever.from_documents(splits)

        doc_results = {}
        for query in queries:
            context_docs = retriever.invoke(query)
            context = chains.invoke({"question": query, "Context": context_docs})
            doc_results[query] = context
            print(f"Document {i}, Query: {query}")
            print(context)
            print("---")

        results[i] = doc_results

    return results

# example
docs_list = [docs[0], docs[1], docs[2], docs[3]]
queries = ["Red flags for lower back pain"]

result_context = process_documents(docs_list, queries)

for doc_num, doc_results in result_context.items():
    print(f"Results for Document {doc_num}:")
    for query, context in doc_results.items():
        print(f"Query: {query}")
        print(context)
        print("---")

#3. The process diverges depending on whether a diagnostic assessment is available.

In [ ]:
# In case there is no diagnostic assessment

import re
from langchain.prompts import PromptTemplate
from langchain_upstage import ChatUpstage

def get_user_input():
    patient_info = {}
    questions = [
        ("patient's chief complaint", "Enter patient's chief complaint: ", lambda x: len(x) > 0),
        ("patient's location", "Enter patient's pain location (e.g. Middle, right): ", lambda x: len(x) > 0),
        ("patient's radiation", "Is there pain radiation? (Yes/No, and location if Yes): ", lambda x: x.lower() in ['yes', 'no'] or (x.lower().startswith('yes') and len(x) > 3)),
        ("patient's severity", "Enter pain severity (mild/moderate/severe): ", lambda x: re.search(r'\b(extremely\s+)?(mild|moderate|severe)\b', x.lower()) is not None),
        ("patient's alleviating factors", "Is pain reduced by lying down? (Yes/No): ", lambda x: x.lower() in ['yes', 'no']),
        ("patient's pain increase", "Pain increase when looking at (aching/opposite/same) side: ", lambda x: x.lower() in ['aching', 'opposite', 'same']),
        ("patient's numbness or tingling", "Numbness or tingling in arm or hand? (Yes/No): ", lambda x: x.lower() in ['yes', 'no']),
        ("patient's weakness", "Weaker or thinner arm than before? (Yes/No): ", lambda x: x.lower() in ['yes', 'no']),
        ("patient's onset of pain", "When did the pain start? ", lambda x: len(x) > 0),
        ("patient's trauma history", "Did pain start within 1 day of trauma? (Yes/No): ", lambda x: x.lower() in ['yes', 'no']),
        ("patient's lower back pain", "Pain also in lower back? (Yes/No): ", lambda x: x.lower() in ['yes', 'no']),
        ("patient's morning stiffness", "Stiffness in morning? (Yes/No): ", lambda x: x.lower() in ['yes', 'no']),
        ("patient's leg symptoms", "Leg weakness or pain? (Yes/No): ", lambda x: x.lower() in ['yes', 'no']),
        ("patient's coronary heart disease history", "History of coronary heart disease? (Yes/No): ", lambda x: x.lower() in ['yes', 'no']),
        ("patient's weight loss/appetite", "Weight loss or decreased appetite? (Yes/No): ", lambda x: x.lower() in ['yes', 'no']),
        ("patient's pregnancy/breastfeeding", "Pregnant or breast feeding? (Yes/No): ", lambda x: x.lower() in ['yes', 'no']),
        ("patient's prolonged sitting", "Prolonged sitting during work? (Yes/No): ", lambda x: x.lower() in ['yes', 'no']),
        ("patient's fever", "Fever? (Yes/No): ", lambda x: x.lower() in ['yes', 'no']),
        ("patient's cancer/steroid history", "History of cancer or steroid use? (Yes/No): ", lambda x: x.lower() in ['yes', 'no']),
        ("patient's osteoporosis", "Osteoporosis? (Yes/No): ", lambda x: x.lower() in ['yes', 'no']),
        ("patient's age", "Patient's age: ", lambda x: x.isdigit() and 0 < int(x) < 120),
        ("patient's alcohol/drug use", "Alcoholic or drug abuse? (Yes/No): ", lambda x: x.lower() in ['yes', 'no']),
        ("patient's HIV status", "HIV? (Yes/No): ", lambda x: x.lower() in ['yes', 'no']),
        ("patient's leg bending difficulty", "Difficult to bend leg? (Yes/No): ", lambda x: x.lower() in ['yes', 'no']),
        ("patient's urinary/fecal incontinence", "Urinary or fecal incontinence? (Yes/No): ", lambda x: x.lower() in ['yes', 'no']),
        ("patient's shoulder drooping or winging", "Shoulder drooping or winging? (Yes/No): ", lambda x: x.lower() in ['yes', 'no']),
        ("patient's upper neck tenderness", "Tenderness at upper neck? (Yes/No): ", lambda x: x.lower() in ['yes', 'no']),
        ("patient's arm lift score", "Arm lift against gravity score (0-5): ", lambda x: x.isdigit() and 0 <= int(x) <= 5),
        ("patient's Babinski Reflex", "Babinski Reflex (positive/negative): ", lambda x: x.lower() in ['positive', 'negative']),
        ("patient's sensation in arms", "Sensation difference between arms? (Yes/No): ", lambda x: x.lower() in ['yes', 'no']),
        ("patient's Spurling test", "Spurling test result (positive/negative): ", lambda x: x.lower() in ['positive', 'negative'])
    ]

    total_questions = len(questions)

    for i, (key, question, validator) in enumerate(questions, 1):
        while True:
            answer = input(question)
            if validator(answer):
                patient_info[key] = answer
                progress = (i / total_questions) * 100
                print(f"Progress: {progress:.1f}%")
                break
            else:
                print("Invalid input. Please try again.")

    return patient_info

def create_prompt_template():
    template = """
    You are a renowned rehabilitation medicine specialist. Check the patient's condition and suggest suspected diagnoses, further evaluations, and red flags based on the condition. Reference the provided pain guides during this process.

    1. Check the patient's condition:
       a) Chief complaint: ${patient's chief complaint}
       b) History taking:
          ${history_questions}
       c) Physical examinations:
          ${physical_exam_questions}

    2. Suggest diagnoses, further evaluations, and search for red flags:
       a) Suggest maximum 3 suspected diagnoses based on history and physical examinations.
          Use the <DIFFERENTIAL DIAGNOSIS> section of the {pain guide}. If no match, suggest "unspecified neck pain".
       b) Suggest further examinations based on symptoms, suspected diagnoses, and the <Imaging and Other Diagnostic Tests> section in {pain guide 2}.
       c) Check for red flags:
          - If present: List red flags from the following list and recommend hospital visit:
            * Fever
            * Unexplained weight loss
            * History of cancer
            * History of violent trauma
            * History of steroid use
            * Osteoporosis
            * Aged younger than 20 years or older than 50 years
            * Failure to improve with treatment
            * History of alcohol or drug abuse
            * HIV
            * Lower extremity spasticity
            * Loss of bowel or bladder function
          - If absent: Suggest rehabilitation exercises from PTX guide where the anatomical structure in the chief complaint is included in <Client's aim>.

    Important notes:
    - Maintain patient confidentiality at all times.
    - Provide your output in a structured format as shown in the examples below.

    ${example_outputs}

    Please provide your assessment and recommendations based on the given information.
    """
    return PromptTemplate.from_template(template)

def create_history_questions():
    return "\n".join([
        "- Location (e.g. upper-lower, left-right): ${location}",
        "- Radiation (include radiating location): ${radiation}",
        "- Severity (severe/moderate/mild): ${severity}",
        "- Pain reduced by recumbency (lying down): ${alleviating_factors}",
        "- More painful when looking at aching side vs opposite side vs same: ${pain_increase}",
        "- Numbness or tingling in arm or hand: ${numbness_tingling}",
        "- Weaker or thinner arm than before: ${weakness}",
        "- When did the pain start: ${onset_of_pain}",
        "- Did the pain start within 1 day of a trauma (e.g. traffic accident, lifting): ${trauma_history}",
        "- Pain also in lower back: ${lower_back_pain}",
        "- Stiffness in morning: ${morning_stiffness}",
        "- Leg weakness or pain: ${leg_symptoms}",
        "- History of coronary heart disease: ${coronary_heart_disease_history}",
        "- Weight loss or decreased appetite: ${weight_loss_appetite}",
        "- Pregnant or breast feeding: ${pregnancy_breastfeeding}",
        "- Prolonged sitting during work: ${prolonged_sitting}",
        "- Fever: ${fever}",
        "- History of cancer or steroid use: ${cancer_steroid_history}",
        "- Osteoporosis: ${osteoporosis}",
        "- Age: ${age}",
        "- Alcoholic or drug abuse: ${alcohol_drug_use}",
        "- HIV: ${hiv_status}",
        "- Difficult to bend leg (leg spasticity): ${leg spasticity}",
        "- Urinary or fecal incontinence: ${urinary_fecal_incontinence}"
    ])

def create_physical_exam_questions():
    return "\n".join([
        "- Shoulder drooping or winging: ${shoulder_drooping_winging}",
        "- Tenderness at upper neck: ${upper_neck_tenderness}",
        "- Arm lift against gravity score (0-5): ${arm_lift_score}",
        "- Babinski Reflex (positive/negative): ${babinski_reflex}",
        "- Sensation difference between arms: ${sensation_in_arms}",
        "- Spurling test result (positive/negative): ${spurling_test}"
    ])

def create_example_outputs():
    return """
    >>Example1<<:
    suspected diagnoses:
    1. Infection (evidence: pain not alleviated by lying down)
    2. Cervical radiculopathy (evidence: numbness and tingling, shoulder drooping, arm lift score ≤3, positive Spurling test)
    3. Brachial plexopathy (evidence: numbness and tingling, upper extremity weakness, shoulder and upper extremity pain)

    further examinations: MRI, Electromyography

    finding red flags: Red flags present: "Fever". Urgent hospital visit recommended.

    >>Example2<<:
    suspected diagnoses:
    1. Muscle strain (evidence: pain increased when turning toward opposite side)
    2. Ankylosing spondylitis (evidence: concurrent lower back pain)

    further examinations: Plain Radiography

    finding red flags: Red flags absent.
    Recommended rehabilitation exercise: "Lifting head off two pillows" - Lie on two pillows, slide your head up the pillow, and nod your head. Gently lift your head off the pillow. Perform slowly and controlled, avoiding pain or other symptoms.

    >>Example3<<:
    suspected diagnoses:
    1. Myelopathy (evidence: Babinski reflex positive, leg weakness or pain, numbness or tingling in arm or hand)
    2. Radiculopathy (evidence: more painful when looking at the aching side, numbess or tingling in arm or hand, trauma)
    3. Cancer (evidence: weakness or pain)

    further examinations: Computed tomography, Electromyography

    finding red flags: Red flags present: "Osteoporosis". Urgent hospital visit recommended.
    """

def Diagnosis_Process():
    # Get user input
    patient_data = get_user_input()

    # Create prompt template and other necessary components
    prompt_template = create_prompt_template()
    history_questions = create_history_questions()
    physical_exam_questions = create_physical_exam_questions()
    example_outputs = create_example_outputs()

    # Create the chain
    llm = ChatUpstage(temperature = 0)
    chain = prompt_template | llm | StrOutputParser()

    # Process documents
    docs_list = [docs[0], docs[1], docs[2], docs[5]]  # You need to define 'docs' somewhere
    chief_complaint = "neck pain"
    queries = [f"symptoms, findings, diagnoses, evaluations related to {chief_complaint}"]
    result_context = process_documents(docs_list, queries)

    # Invoke the chain
    result = chain.invoke({
        "history_questions": history_questions,
        "physical_exam_questions": physical_exam_questions,
        "example_outputs": example_outputs,
        "pain guide": result_context[0],
        "pain guide 2": result_context[1],
        "pain guide 3": result_context[2],
        "PTX": result_context[3],
        **patient_data
    })

    print(result)

In [ ]:
# In case there is a diagnostic assessment

import pandas as pd
from datetime import datetime, timedelta

def calculate_7day_average(file_path, date=None):
    """
    Calculates the average of data within 7 days from the specified date in the given CSV file.

    :param file_path: Path to the CSV file
    :param date: Reference date (default: None, uses today's date if None)
    :return: List of 7-day average values for each item
    """
    df = pd.read_csv(file_path, parse_dates=['datetime'])

    if date is None:
        date = datetime.now()
    elif isinstance(date, str):
        date = datetime.strptime(date, "%Y-%m-%d")

    seven_days_ago = date - timedelta(days=7)

    df_recent = df[df['datetime'] > seven_days_ago]

    averages = []
    for item in range(1, 18):
        column_name = f'Item {item}'
        avg = df_recent[column_name].mean()
        averages.append(round(avg, 2))

    return averages

def input_item_scores():
    """
    Function to input scores for Items 1 to 17 from the user.

    :return: List containing 17 item scores
    """
    scores = []
    for i in range(1, 18):
        while True:
            try:
                score = float(input(f"Enter the score for Item {i}: "))
                scores.append(score)
                break
            except ValueError:
                print("Please enter a valid number.")

    return scores

def compare_scores(item_scores, average_scores):
    """
    Compare current scores with 7-day averages and identify items with decreased scores.

    :param item_scores: List of current scores for items 1-17
    :param average_scores: List of 7-day average scores for items 1-17
    :return: List of tuples containing (item name, current score, average score) for decreased items
    """
    item_names = [
        "Reach fwd", "Reach Up", "Reach Down", "Lift Up", "Push Down",
        "Wrist Up", "Acquire - Release", "Grasp Dynamometer", "Lateral Pinch",
        "Pull Weight", "Push Weight", "Container", "Pinch Die", "Pencil",
        "Manipulate (chip)", "Push Index", "Push Thumb"
    ]

    decreased_items = []

    for i, (current, average) in enumerate(zip(item_scores, average_scores)):
        if current < average:
            decreased_items.append(item_names[i])

    return decreased_items

# Function to get patient information from user input
def get_patient_info():
    # Prompt user for each piece of information
    diagnosed_patient = input('chief_complaint: ')
    patient_disability = input("Patient's disability (e.g., Activities using arm): ")
    functional_evaluation = input("Functional evaluation tool (e.g., CUE-T): ")
    new_symptoms = input("Newly acquired symptoms: ")

    # Return the information as a list
    return [
        patient_disability,
        functional_evaluation,
        new_symptoms
    ]

def rehabilitation_evaluation(llm, docs, decreased_items, patient_info):
    prompt_template = PromptTemplate.from_template(
        """
        You are a renowned rehabilitation medicine specialist. Evaluate physical functions related to patient's diagnosis and disabilities. Educate the patient with proper rehabilitation exercise with regards to functions declining over time. Check whether there are recently acquired symptoms and check whether those symptoms indicate complications related to patient's diagnosis.

        Functional evaluation and exercise education consists of 3 steps. Answer should be provided in 3 steps.

        - $1 Based on {patient's disability}, suggest which {functional evaluation} is needed for the patient.

        - $2 Suggest body parts used during each item in {ITEMs} with <INTENT> section in {CUE T Manual}.

        - $3 Suggest exercises in {PTX} where "anatomical structures used during each item in {ITEMs}.


        - $4 Get {diagnosed patient} as diagnosis. Check if diagnosis is "Stroke" or "Spinal Cord Injury".
             Get {newly acquired symptoms} as "symptoms".
             - If diagnosis is "Stroke", Show list of extracted complications to patient and recommend hospital visit if "symptoms" indicate certain complications among complications extracted from {Stroke Complications}. Give evidence why "symptoms" indicate extracted complication.
             - Else if diagnosis is "Spinal Cord Injury", Show list of extracted complications to patient and recommend hospital visit if "symptoms" indicate certain complications among complications extracted from {SCI Complications}. Give evidence why "symptoms" indicate extracted complication.

        ---
        Here are the examples of questions and answers between physician and patients.
        ---
        >>Example1<< :

        $Input :
            "diagnosed patient": 'Stroke',
            "patient's disability": 'Activities using arm',
            "functional evaluation": 'CUE-T',
            "ITEMs": ['REACH FORWARD'],
            "newly acquired symptoms":'Pain when moving my weak side shoulders'

        $output :
            required test : CUE T Test.
            Extract anatomical structures : shoulder.

            Item of which the score dropped : 'Reach Forward'
            Exercise recommendation :
            1. Position yourself standing against a wall.
            2. Take your arm out of the sling, lean forward and allow the affected arm to passively flex with gravity.
            3. Use your other arm to slowly lift the shoulder into flexion.
            4. Ensure that you do not actively use the affected arm.

            suspected complications: hemiplegic shoulder pain (because symptom is pain on movement (active or passive))
            Visit nearby hospital: Yes


        >>Example2<< :

        $Input :
            "diagnosed patient": 'Spinal Cord Injury',
            "patient's disability": 'Activities using arm',
            "functional evaluation": 'CUE-T',
            "ITEMs": ['PINCH DIE', 'WRIST UP'],
            "newly acquired symptoms":'Face looks pale, frequently feel dizzy'

        $output :
            required test : CUE T Test.
            Extract anatomical structures : fingers, wrist.


            Item of which the score dropped : 'Pinch Die'
            Exercise recommendation :
            1. Position yourself with your fingers weaved between each other.
            2. Push up with the fingers of your right hand while pushing down with the fingers of your left hand.

            Item of which the score dropped : 'Wrist up'
            Exercise recommendation :
            1. Position yourself sitting with your hand grasping a cup and hanging over the edge of a table.
            2. Practice tilting the cup up by bending your wrist to a point level with, or higher than the table.

            suspected complications: Orthostatic hypotension (because symptoms are pallor and dizziness)
            Visit nearby hospital: Yes
        ---

        """
    )
    chain = prompt_template | llm | StrOutputParser()

    docs_list = [docs[3], docs[5], docs[6], docs[7]]

    diagnosed_patient = patient_info[0]
    queries = [f"upper extremity, complications"]
    result_context = process_documents(docs_list, queries)

    result = chain.invoke({
        "CUE T Manual": result_context[0],
        "PTX": result_context[1],
        "Stroke Complications": result_context[2],
        "SCI Complications": result_context[3],
        "diagnosed patient": diagnosed_patient,
        "patient's disability": patient_info[0],
        "functional evaluation": patient_info[1],
        "ITEMs": decreased_items,
        "newly acquired symptoms": patient_info[2]
    })

    return result

def rehabilitation_assessment_workflow(file_path):
    # Step 1: Calculate 7-day average
    average_scores = calculate_7day_average(file_path)
    print("7-day average scores calculated.")

    # Step 2: Input current item scores
    print("\nPlease input the current scores for each item:")
    current_scores = input_item_scores()

    # Step 3: Compare scores and identify decreased items
    decreased_items = compare_scores(current_scores, average_scores)
    print("\nItems with decreased scores:", decreased_items)

    # Step 4: Get patient information
    print("\nPlease provide the following patient information:")
    patient_info = get_patient_info()

    result = rehabilitation_evaluation(llm, docs, decreased_items, patient_info)

    print("\nAssessment Result:")
    print(result)

    print("\nWorkflow completed.")


In [ ]:
def check_diagnosis():
    has_diagnosis = input("Do you have a diagnostic assessment? (yes/no): ").lower()

    if has_diagnosis == "yes":
        diagnosis_id = input("Please enter the diagnostic assessment ID: ")
        file_path = f'/content/drive/MyDrive/240814_Llama_RAG/{diagnosis_id}.csv'
        rehabilitation_assessment_workflow(llm, docs, file_path)
    elif has_diagnosis == "no":
        Diagnosis_Process()
    else:
        print("Invalid input. Please answer 'yes' or 'no'.")
        check_diagnosis()  # Recursively call the function to get correct input

# Execute the function
check_diagnosis()